In [1]:
from math import gcd,log
from random import randint
import numpy as np
from qiskit import *

from qiskit.primitives import BackendSamplerV2
from qiskit.providers.basic_provider import BasicProvider
from qiskit_aer import AerSimulator
from qiskit_aer.primitives import Sampler

import RSA_module

# To get the qasm_simulator directly
qasm_sim = AerSimulator()
# qasm_sim = BasicProvider().get_backend("qasm_simulator")

bit_length = 4

public, private = RSA_module.generate_keypair(2**bit_length)
print("\nChave pública: " + str(public))
print("Chave privada: " + str(private))


p: 197, q: 211

Chave pública: (23179, 41567)
Chave privada: (38539, 41567)


In [2]:
msg = "Olá mundo!"
encrypted_msg, encryption_obj = RSA_module.encrypt(msg, public)

print("\nMensagem original: " + msg)
print("Mensagem encriptada: " + encrypted_msg)


Mensagem original: Olá mundo!
Mensagem encriptada: 28865275971161930974230388303189433022982522945


In [3]:
decrypted_msg = RSA_module.decrypt(encryption_obj, private)

print("\nMensagem decriptada: " + decrypted_msg)


Mensagem decriptada: Olá mundo!


In [4]:
def period(a,N):
    available_qubits = 16 
    r=-1   
    
    if N >= 2**available_qubits:
        print(str(N)+' is too big for IBMQX')
    
    qr = QuantumRegister(available_qubits)   
    cr = ClassicalRegister(available_qubits)
    qc = QuantumCircuit(qr,cr)
    x0 = randint(1, N-1) 
    x_binary = np.zeros(available_qubits, dtype=bool)
    
    for i in range(1, available_qubits + 1):
        bit_state = (N%(2**i)!=0)
        if bit_state:
            N -= 2**(i-1)
        x_binary[available_qubits-i] = bit_state    
    
    for i in range(0,available_qubits):
        if x_binary[available_qubits-i-1]:
            qc.x(qr[i])
    x = x0
    
    while np.logical_or(x != x0, r <= 0):
        r+=1
        qc.measure(qr, cr) 
        for i in range(0,3): 
            qc.x(qr[i])
        qc.cx(qr[2],qr[1])
        qc.cx(qr[1],qr[2])
        qc.cx(qr[2],qr[1])
        qc.cx(qr[1],qr[0])
        qc.cx(qr[0],qr[1])
        qc.cx(qr[1],qr[0])
        qc.cx(qr[3],qr[0])
        qc.cx(qr[0],qr[1])
        qc.cx(qr[1],qr[0])
        
        sampler = Sampler()
        job = sampler.run(qc, shots=1024)
        result = job.result()
        quasi_dist = result.quasi_dists[0]
        total_shots = result.metadata[0]['shots'] # Get total shots from metadata
        counts = {}
        for bitstring, prob in quasi_dist.items():
            count = round(prob * total_shots)
            if count > 0: # Only include outcomes with non-zero counts
                counts[bitstring] = count
        print(qc)
        results = [[],[]]
        for key,value in counts.items(): #the result should be deterministic but there might be some quantum calculation error so we take the most reccurent output
            results[0].append(key)
            results[1].append(int(value))
        s = results[0][np.argmax(np.array(results[1]))]
    return r

In [5]:
def shors_breaker(N):
    N = int(N)
    while True:
        a=randint(0,N-1)
        g=gcd(a,N)
        if g!=1 or N==1:
            return g,N//g
        else:
            r=period(a,N) 
            if r % 2 != 0:
                continue
            elif pow(a,r//2,N)==-1:
                continue
            else:
                p=gcd(pow(a,r//2)+1,N)
                q=gcd(pow(a,r//2)-1,N)
                if p==N or q==N:
                    continue
                return p,q

In [6]:
def modular_inverse(a,m): 
    a = a % m; 
    for x in range(1, m) : 
        if ((a * x) % m == 1) : 
            return x 
    return 1

In [7]:
N=18727
assert N>0,"Input must be positive"
print(shors_breaker(N))


/tmp/ipykernel_56577/1352048160.py:9: DeprecationWarning: Sampler has been deprecated as of Aer 0.15, please use SamplerV2 instead.
  r=period(a,N)


       ┌───┐                        ┌─┐┌───┐                         ┌───┐     »
 q0_0: ┤ X ├────────────────────────┤M├┤ X ├─────────────────────────┤ X ├──■──»
       ├───┤                        └╥┘└┬─┬┘┌───┐     ┌───┐     ┌───┐└─┬─┘┌─┴─┐»
 q0_1: ┤ X ├─────────────────────────╫──┤M├─┤ X ├─────┤ X ├──■──┤ X ├──■──┤ X ├»
       ├───┤                         ║  └╥┘ └┬─┬┘┌───┐└─┬─┘┌─┴─┐└─┬─┘     └───┘»
 q0_2: ┤ X ├─────────────────────────╫───╫───┤M├─┤ X ├──■──┤ X ├──■────────────»
       └───┘┌─┐                      ║   ║   └╥┘ └───┘     └───┘               »
 q0_3: ─────┤M├──────────────────────╫───╫────╫────────────────────────────────»
            └╥┘┌─┐                   ║   ║    ║                                »
 q0_4: ──────╫─┤M├───────────────────╫───╫────╫────────────────────────────────»
       ┌───┐ ║ └╥┘                   ║   ║    ║   ┌─┐                          »
 q0_5: ┤ X ├─╫──╫────────────────────╫───╫────╫───┤M├──────────────────────────»
       └───┘ ║  ║ ┌─┐       

/tmp/ipykernel_56577/1352048160.py:9: DeprecationWarning: Sampler has been deprecated as of Aer 0.15, please use SamplerV2 instead.
  r=period(a,N)


       ┌───┐                        ┌─┐┌───┐                         ┌───┐     »
 q1_0: ┤ X ├────────────────────────┤M├┤ X ├─────────────────────────┤ X ├──■──»
       ├───┤                        └╥┘└┬─┬┘┌───┐     ┌───┐     ┌───┐└─┬─┘┌─┴─┐»
 q1_1: ┤ X ├─────────────────────────╫──┤M├─┤ X ├─────┤ X ├──■──┤ X ├──■──┤ X ├»
       ├───┤                         ║  └╥┘ └┬─┬┘┌───┐└─┬─┘┌─┴─┐└─┬─┘     └───┘»
 q1_2: ┤ X ├─────────────────────────╫───╫───┤M├─┤ X ├──■──┤ X ├──■────────────»
       └───┘┌─┐                      ║   ║   └╥┘ └───┘     └───┘               »
 q1_3: ─────┤M├──────────────────────╫───╫────╫────────────────────────────────»
            └╥┘┌─┐                   ║   ║    ║                       ┌─┐      »
 q1_4: ──────╫─┤M├───────────────────╫───╫────╫───────────────────────┤M├──────»
       ┌───┐ ║ └╥┘                   ║   ║    ║   ┌─┐                 └╥┘      »
 q1_5: ┤ X ├─╫──╫────────────────────╫───╫────╫───┤M├──────────────────╫───────»
       └───┘ ║  ║ ┌─┐       

In [8]:
N_shor = public[1]
assert N_shor>0,"Input must be positive"
p,q = shors_breaker(N_shor)
phi = (p-1) * (q-1)  
d_shor = modular_inverse(public[0], phi) 

        ┌───┐                     ┌─┐┌───┐                         ┌───┐     »
 q25_0: ┤ X ├─────────────────────┤M├┤ X ├─────────────────────────┤ X ├──■──»
        ├───┤                     └╥┘└┬─┬┘┌───┐     ┌───┐     ┌───┐└─┬─┘┌─┴─┐»
 q25_1: ┤ X ├──────────────────────╫──┤M├─┤ X ├─────┤ X ├──■──┤ X ├──■──┤ X ├»
        ├───┤                      ║  └╥┘ └┬─┬┘┌───┐└─┬─┘┌─┴─┐└─┬─┘     └───┘»
 q25_2: ┤ X ├──────────────────────╫───╫───┤M├─┤ X ├──■──┤ X ├──■────────────»
        ├───┤                      ║   ║   └╥┘ └┬─┬┘     └───┘               »
 q25_3: ┤ X ├──────────────────────╫───╫────╫───┤M├──────────────────────────»
        ├───┤                      ║   ║    ║   └╥┘  ┌─┐                     »
 q25_4: ┤ X ├──────────────────────╫───╫────╫────╫───┤M├─────────────────────»
        └───┘┌─┐                   ║   ║    ║    ║   └╥┘                     »
 q25_5: ─────┤M├───────────────────╫───╫────╫────╫────╫──────────────────────»
        ┌───┐└╥┘                   ║   ║    ║    ║  

/tmp/ipykernel_56577/1352048160.py:9: DeprecationWarning: Sampler has been deprecated as of Aer 0.15, please use SamplerV2 instead.
  r=period(a,N)


        ┌───┐                     ┌─┐┌───┐                         ┌───┐     »
 q26_0: ┤ X ├─────────────────────┤M├┤ X ├─────────────────────────┤ X ├──■──»
        ├───┤                     └╥┘└┬─┬┘┌───┐     ┌───┐     ┌───┐└─┬─┘┌─┴─┐»
 q26_1: ┤ X ├──────────────────────╫──┤M├─┤ X ├─────┤ X ├──■──┤ X ├──■──┤ X ├»
        ├───┤                      ║  └╥┘ └┬─┬┘┌───┐└─┬─┘┌─┴─┐└─┬─┘     └───┘»
 q26_2: ┤ X ├──────────────────────╫───╫───┤M├─┤ X ├──■──┤ X ├──■────────────»
        ├───┤                      ║   ║   └╥┘ └┬─┬┘     └───┘               »
 q26_3: ┤ X ├──────────────────────╫───╫────╫───┤M├──────────────────────────»
        ├───┤                      ║   ║    ║   └╥┘  ┌─┐                     »
 q26_4: ┤ X ├──────────────────────╫───╫────╫────╫───┤M├─────────────────────»
        └───┘┌─┐                   ║   ║    ║    ║   └╥┘                     »
 q26_5: ─────┤M├───────────────────╫───╫────╫────╫────╫──────────────────────»
        ┌───┐└╥┘                   ║   ║    ║    ║  

In [9]:
decrypted_msg = RSA_module.decrypt(encryption_obj, (d_shor,N_shor))

print('\nMensagem quebrada com o algorítmo de Shor: ' + decrypted_msg + "\n")


Mensagem quebrada com o algorítmo de Shor: Olá mundo!

